In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

In [2]:
# Conflict Data - admin1 (State)/admin2 (County)
# Health Facilies - state/county
# Migration - admin1(State)/admin2 (County) - Population
# Demographics - Just Demographics (State/County)

### Cleaning Conflict Data
Source: </br>
https://acleddata.com/data-export-tool/ <br>
All conflict data was taken from the Armed Conflict Location and Event Data project (ACLED) using their data export tool for all events from decemember 2011 through december 2015.

In [3]:
# Event date, year, admin1, admin2, fatalities 
# Plan to slice down to year county/fatalities
conflict = pd.read_csv('../00_source/conflict_data.csv')

# Slicing Conflict Data
conflict_sliced = conflict[['event_date', 'year', 'admin1', 'admin2', 'fatalities']]

In [4]:
conflict_sliced.head()

,event_date,year,admin1,admin2,fatalities
0,31 December 2015,2015,Western Bahr el Ghazal,Wau,0
1,30 December 2015,2015,Western Bahr el Ghazal,Jur River,5
2,30 December 2015,2015,Western Equatoria,Yambio,0
3,28 December 2015,2015,Western Equatoria,Yambio,0
4,28 December 2015,2015,Western Bahr el Ghazal,Wau,1


In [5]:
# Validating representation (76 of 79 Counties.)
len(conflict_sliced['admin2'].unique())

76

In [12]:
# Validating this Dataset has information in the correct years 
conflict_sliced['year'].unique()

array([2015, 2014, 2013, 2012, 2011], dtype=int64)

In [13]:
# Grouping by year/region
conflict_sliced_grouped = conflict_sliced.groupby(['admin2', 'year']).sum()

#conflict_sliced_grouped['year'].unique()
conflict_sliced_grouped.to_csv('../20_intermediate/aggregated_conflict_data.csv')

# Reading out to CSV then back in fixes index
merged_conflict_data = pd.read_csv('../20_intermediate/aggregated_conflict_data.csv')

# Reordering columns
merged_conflict_data = merged_conflict_data[['year', 'admin2', 'fatalities']]

# Printing the DF for Merging
merged_conflict_data.head()

,year,admin2,fatalities
0,2012,Abiemnhom,7
1,2013,Abiemnhom,8
2,2014,Abiemnhom,0
3,2011,Akobo,1
4,2012,Akobo,98


### Hospital Data 
Source: </br>
http://www.southsudanmedicaljournal.com/assets/files/Journals/vol_12_iss_2_mar_19/SSMJ%2012%202%20Issue%20Final.pdf

In [14]:
# Note nothing on hospital size or construction date in the DF
hospitals = pd.read_excel('../00_source/health_facilities.xls')

In [15]:
# PHCU - Primary Health Care Unit
# PHCC - Primary Health Care Centers
hospitals['HF_TYPE'].value_counts()

PHCU                    1079
PHCC                     306
County Hospital           28
Private Clinic            15
Specialized Hospital      13
Other                     10
State Hospital             8
Teaching Hospital          3
Hospital                   1
Name: HF_TYPE, dtype: int64

In [16]:
print("There are {} medical facilities in South Sudan in {} of the 79 counties.".format(hospitals.shape[0],len(hospitals['COUNTY'].unique())))
print("This dataset does not contain information on size or construction date of the hospitals.")

There are 1463 medical facilities in South Sudan in 77 of the 79 counties.
This dataset does not contain information on size or construction date of the hospitals.


In [17]:
# Filtering hospitals
# 1463 Total Entries, 784 NaN, 648 Functional, 31 Non Fuctional. Assumed NAs were functional   
hospitals['FUNCTIONAL Status'].value_counts()

Functional        648
Non Functional     31
Name: FUNCTIONAL Status, dtype: int64

In [18]:
# Removing Non Functional Hospitals
hospitals_filtered = hospitals.loc[~(hospitals['FUNCTIONAL Status'] == 'Non Functional')]

# To Avoid Copy/View Warning.
hospitals_filtered = hospitals_filtered.copy() 

# Replacing all of the NaNs with Functional
hospitals_filtered['FUNCTIONAL Status'] = hospitals_filtered['FUNCTIONAL Status'].fillna(value='Functional')

# Dropping ID and source
del hospitals_filtered['ID']
del hospitals_filtered['Source']

hospitals_filtered.head(3)

,STATE,COUNTY,PAYAM,FACIL_NAME,FACIL_NAME2,LATITUDE,LONGITUDE,Q_CODE,HF_CODE,HF_TYPE,FUNCTIONAL Status
0,Central Equatoria,Juba,Juba Town,AIC BULUK,Aic Buluk,4.84718,31.59355,92093,15,PHCU,Functional
1,Central Equatoria,Juba,Lokiliri,ARU HEALTH UNIT,Aru Health Unit,4.36354,31.98460,92235,15,PHCU,Functional
2,Central Equatoria,Juba,Juba Town,BELPHAM MEDICAL HQ,Belpham Medical Hq,4.81890,31.57712,92206,14,PHCC,Functional


In [19]:
# Slicing the hospital
hospitals_filtered = hospitals_filtered[['STATE', 'COUNTY', 'FACIL_NAME2', 'HF_CODE', 'HF_TYPE']]

# Getting a count of hospitals by region. 
count_total = hospitals_filtered.groupby('COUNTY')['FACIL_NAME2'].count().rename("county_hospitals").reset_index()

# Combining back together
hospitals_filtered_added =  hospitals_filtered.merge(count_total)

# Dropping duplicates 
hospitals_county_level = hospitals_filtered_added.drop_duplicates('COUNTY')

# Slicing to just state county and hospital number
hospitals_county_level = hospitals_county_level[['STATE', 'COUNTY','county_hospitals']]
hospitals_county_level.head()

,STATE,COUNTY,county_hospitals
0,Central Equatoria,Juba,76
76,Central Equatoria,Kajo-Keji,46
122,Central Equatoria,Lainya,33
155,Central Equatoria,Morobo,12
167,Central Equatoria,Terekeka,45


In [20]:
print("Even after these operations the dataframe still retained the original {} counties.".format(len(hospitals_county_level['COUNTY'].unique())))

Even after these operations the dataframe still retained the original 77 counties.


### Merging Conflict and Hospital Data

In [21]:
# Hospitals and Conflict Merge
hos_conf_merged = pd.merge(merged_conflict_data, hospitals_county_level, left_on='admin2', right_on='COUNTY',how='left')

# Renaming 
hos_conf_merged = hos_conf_merged.rename(columns={'STATE':'state','COUNTY':'county'})

# Reordering Dataframe
hos_conf_merged = hos_conf_merged[['year', 'state', 'county','fatalities', 'county_hospitals']]

# Checking the DF
hos_conf_merged.head()

,year,state,county,fatalities,county_hospitals
0,2012,Unity,Abiemnhom,7,5.0
1,2013,Unity,Abiemnhom,8,5.0
2,2014,Unity,Abiemnhom,0,5.0
3,2011,Jonglei,Akobo,1,14.0
4,2012,Jonglei,Akobo,98,14.0


### Demographics
Population: </br>
https://www.worldometers.info/world-population/south-sudan-population/

In [22]:
# Reading in Data 
demographics = pd.read_excel('../00_source/demographics_2020.xlsx')

# Printing the head
demographics.head(3)

,State,Pcode1,County,Pcode2,2019 Baseline Population,Children <18yrs,No. male children under 5,No. female children under 5,No. male children aged 5 - 17 years,No. female children aged 5 - 17 years,No. male adults aged 18 - 60,No. female adults aged 18 - 60,No. male adults aged over 60,No. female adults aged over 60
0,Central Equatoria,92.0,Juba,9202.0,499537.632115,241976.028996,36965.784777,48654.965368,76928.795346,79426.483506,115892.730651,120388.569340,11739.134355,9541.168773
1,Central Equatoria,92.0,Kajo-keji,9206.0,221902.149132,35948.148159,3617.005031,6679.254689,12759.373575,12892.514865,72783.904915,71008.687722,28181.572940,13979.835395
2,Central Equatoria,92.0,Lainya,9203.0,110281.690307,46318.309929,11579.577482,11138.450721,12130.985934,11469.295792,31540.563428,25805.915532,2977.605638,3639.295780


In [23]:
print("This DF has one extra county {} instead of 79.".format(len(demographics['County'].unique())))

This DF has one extra county 80 instead of 79.


In [25]:
print("This dataset represents {} of the 79 counties".format(len(migration_sliced['County'].unique())))

This dataset represents 73 of the 79 counties


### Population

In [26]:
population = pd.read_excel('../00_source/population.xlsx')

# Printing the head
population.head()

,Admin_2,admin2Pcod,admin2RefN,Admin_1,SS2008,WP2010,LS2011,LS2012,LS2013,LS2014,WP2015,UN2014,ETHIOPIA,UGANDA,"IDP - Dec. 3, 2015",Internal Out-Migration,Refugees,Net Migration,Estimated 2015 Population,PopChange
0,Abiemnhom,SS1302,Abiemnhom,Unity,17012.0,22025,19393,21754.0,23203.0,24080,31683,23369,0,0,14217,11735.715071,0,2481.284929,25850.284929,0.106178
1,Akobo,SS1207,Akobo,Jonglei,136210.0,132141,157474,176578.0,184131.0,191996,188273,170211,22885,0,34387,45815.782100,22885,-11428.782100,135897.217900,-0.201596
2,Aweil Centre,SS2205,Aweil Centre,Northern Bahr el Ghazal,41827.0,58488,49174,55547.0,56193.0,57442,87394,106523,0,0,0,0.000000,0,0.000000,106523.000000,0.000000
3,Aweil East,SS2202,Aweil East,Northern Bahr el Ghazal,309921.0,299846,353806,398384.0,417754.0,436824,431148,529098,0,0,0,0.000000,0,0.000000,529098.000000,0.000000
4,Aweil North,SS2201,Aweil North,Northern Bahr el Ghazal,129127.0,142807,146303,164515.0,174104.0,184654,206227,267215,0,0,0,0.000000,0,0.000000,267215.000000,0.000000


In [36]:
population_slice = population[['Admin_2','Admin_1','LS2011','LS2012','LS2013','LS2014','WP2015']]

population_slice.columns = ['county', 'state', '2011','2012', '2013', '2014', '2015']

population_slice.head()

,county,state,2011,2012,2013,2014,2015
0,Abiemnhom,Unity,19393,21754.0,23203.0,24080,31683
1,Akobo,Jonglei,157474,176578.0,184131.0,191996,188273
2,Aweil Centre,Northern Bahr el Ghazal,49174,55547.0,56193.0,57442,87394
3,Aweil East,Northern Bahr el Ghazal,353806,398384.0,417754.0,436824,431148
4,Aweil North,Northern Bahr el Ghazal,146303,164515.0,174104.0,184654,206227


In [38]:
population_melt = population_slice.melt(id_vars=['county', 'state'])
population_melt.columns = ['county', 'state', 'year', 'population']
population_melt.head()

,county,state,year,population
0,Abiemnhom,Unity,2011,19393.0
1,Akobo,Jonglei,2011,157474.0
2,Aweil Centre,Northern Bahr el Ghazal,2011,49174.0
3,Aweil East,Northern Bahr el Ghazal,2011,353806.0
4,Aweil North,Northern Bahr el Ghazal,2011,146303.0


In [52]:
population_melt['year'] = population_melt['year'].astype('int64')
hos_conf_merged['year'] = hos_conf_merged['year'].astype('int64')

In [53]:
# merge with fatalities / hospital data

death_pop_hosp = pd.merge(hos_conf_merged, population_melt, on= ['county', 'state', 'year'])
death_pop_hosp.head()

,year,state,county,fatalities,county_hospitals,population
0,2012,Unity,Abiemnhom,7,5.0,21754.0
1,2013,Unity,Abiemnhom,8,5.0,23203.0
2,2014,Unity,Abiemnhom,0,5.0,24080.0
3,2011,Jonglei,Akobo,1,14.0,157474.0
4,2012,Jonglei,Akobo,98,14.0,176578.0


In [54]:
# get fatalities per 10k
death_pop_hosp["intensity"] = death_pop_hosp['fatalities'] / death_pop_hosp['population'] * 10000
death_pop_hosp.head()

,year,state,county,fatalities,county_hospitals,population,intensity
0,2012,Unity,Abiemnhom,7,5.0,21754.0,3.217799
1,2013,Unity,Abiemnhom,8,5.0,23203.0,3.447830
2,2014,Unity,Abiemnhom,0,5.0,24080.0,0.000000
3,2011,Jonglei,Akobo,1,14.0,157474.0,0.063503
4,2012,Jonglei,Akobo,98,14.0,176578.0,5.549955


In [55]:
death_pop_hosp.to_csv('../20_intermediate/death_pop.csv')